In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
pd.set_option("display.max_rows", 101)
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 15
import seaborn as sns
from collections import Counter
from PIL import Image
import math
import seaborn as sns
from collections import defaultdict
from pathlib import Path
import cv2
from tqdm import tqdm

In [2]:
import tensorflow as tf
from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split

/home/pranjut/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
STEEL_PATH="dataset"
TRAIN_CSV = "train.csv"
SUBMIT_SAMPLE = "sample_submission.csv"
TRAIN_IMAGE=STEEL_PATH + "/train_images"
TEST_IMAGE=STEEL_PATH + "/test_images/"
PRETRAINED_MODEL=STEEL_PATH + '/saved_model/ResNet_network.h5'

In [4]:
def conv_block(x, filters, kernel_size=3, padding='same', strides=1):
    conv = tf.keras.layers.Activation('relu')(x)
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

In [5]:
def residual_block(x, filters, kernel_size=3, padding='same', strides=1):
    res = conv_block(x, filters, 3, padding, strides)
    res = conv_block(res, filters, 3, padding, 1)
    skip = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    skip = tf.keras.layers.BatchNormalization()(skip)
    output = Add()([skip, res])
    return output

In [6]:
def bn_act(x, act=True):
    'batch normalization layer with an optinal activation layer'
    x = tf.keras.layers.BatchNormalization()(x)
    if act == True:
        x = tf.keras.layers.Activation('relu')(x)
    return x

In [7]:
def stem(x, filters, kernel_size=3, padding='same', strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size, padding, strides)
    shortcut = Conv2D(filters, kernel_size=1, padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    output = Add()([conv, shortcut])
    return output

In [8]:
def upsample_concat_block(x, xskip):
    u = UpSampling2D((2,2))(x)
    c = Concatenate()([u, xskip])
    return c

In [9]:
def network(img_h, img_w):
    f = [16, 32, 64, 128, 256]
    inputs = Input((img_h, img_w, 1))
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = tf.keras.layers.Conv2D(4, (1, 1), padding="same", activation="sigmoid")(d4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

In [10]:
def tversky(y_true, y_pred, smooth=1e-6):
    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def focal_tversky_loss(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return tf.keras.backend.pow((1-pt_1), gamma)

In [11]:
img_w = 800
img_h = 256
model = network(img_h=img_h, img_w=img_w)
adam = tf.keras.optimizers.Adam(lr = 0.05, epsilon = 0.1)
model.compile(optimizer=adam, loss=focal_tversky_loss, metrics=[tversky])

In [12]:
# model.load_weights(PRETRAINED_MODEL)

In [13]:
def load_data(steel_path=STEEL_PATH, file_name=TRAIN_CSV):
    csv_path = os.path.join(steel_path, file_name)
    return pd.read_csv(csv_path).fillna(-1)

In [14]:
def rle_to_mask(rle_string,height,width):
        '''
        convert RLE(run length encoding) string to numpy array

        Parameters: 
        rleString (str): Description of arg1 
        height (int): height of the mask
        width (int): width of the mask 

        Returns: 
        numpy.array: numpy array of the mask
        '''
        rows, cols = height, width
        if rle_string == -1:
            return np.zeros((height, width))
        else:
            rleNumbers = [int(numstring) for numstring in rle_string.split(' ')]
            rlePairs = np.array(rleNumbers).reshape(-1,2)
            img = np.zeros(rows*cols,dtype=np.uint8)
            for index,length in rlePairs:
                index -= 1
                img[index:index+length] = 255
            img = img.reshape(cols,rows)
            img = img.T
            return img

In [15]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_ids, labels, image_dir, batch_size=32,
                 img_h=256, img_w=512, shuffle=True):
        
        self.list_ids = list_ids
        self.labels = labels
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.img_h = img_h
        self.img_w = img_w
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        'denotes the number of batches per epoch'
        return int(np.floor(len(self.list_ids)) / self.batch_size)
    
    def __getitem__(self, index):
        'generate one batch of data'
        print("Generating data .....")
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # get list of IDs
        list_ids_temp = [self.list_ids[k] for k in indexes]
        # generate data
        X, y = self.__data_generation(list_ids_temp)
        # return data 
        return X, y
    
    def on_epoch_end(self):
        'update ended after each epoch'
        self.indexes = np.arange(len(self.list_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_ids_temp):
        'generate data containing batch_size samples'
        X = np.empty((self.batch_size, self.img_h, self.img_w, 1))
        y = np.empty((self.batch_size, self.img_h, self.img_w, 4))
        
        for idx, id in enumerate(list_ids_temp):
            file_path =  os.path.join(self.image_dir, id)
            image = cv2.imread(file_path, 0)
            image_resized = cv2.resize(image, (self.img_w, self.img_h))
            image_resized = np.array(image_resized, dtype=np.float64)
            # standardization of the image
            image_resized -= image_resized.mean()
            image_resized /= image_resized.std()
            
            mask = np.empty((img_h, img_w, 4))
            
            for idm, image_class in enumerate(['1','2','3','4']):
                rle = self.labels.get(id + '_' + image_class)
                # if there is no mask create empty mask
                if rle is None:
                    class_mask = np.zeros((1600, 256))
                else:
                    class_mask = rle_to_mask(rle, width=1600, height=256)
             
                class_mask_resized = cv2.resize(class_mask, (self.img_w, self.img_h))
                mask[...,idm] = class_mask_resized
            
            X[idx,] = np.expand_dims(image_resized, axis=2)
            y[idx,] = mask
        
        # normalize Y
        y = (y > 0).astype(int)     
        return X, y

In [16]:
train_df = load_data()
image_ids_with_duplicats = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
# print(type(image_ids_with_duplicats))
image_ids = image_ids_with_duplicats.unique()
# train_df["ImageId_ClassId"].head()
# train_df.head()

In [17]:
X_train, X_rest = train_test_split(image_ids, test_size=.30, random_state=42)
# X_val, X_test = train_test_split(X_rest, test_size=.50, random_state=42)

len(X_train)

8797

In [18]:
params = {'img_h': img_h,
          'img_w': img_w,
          'image_dir': TRAIN_IMAGE,
          'batch_size': 12,
          'shuffle': True}

labels = {}
# print(labels)
for index, row in train_df[train_df['EncodedPixels']!=-1].iterrows():
    labels[row['ImageId_ClassId']] = row['EncodedPixels']
    

# for key in labels.keys():
#     print(labels[key])

# Get Generators
training_generator = DataGenerator(X_train, labels, **params)
validation_generator = DataGenerator(X_rest, labels, **params)

X_train


array(['33514c0b1.jpg', '2d1658337.jpg', 'fbf282a60.jpg', ...,
       '6cf49f691.jpg', '11b1e2910.jpg', '93c5acf9d.jpg'], dtype=object)

In [ ]:
history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=3, verbose=1)

Epoch 1/3
